In [1]:
import urllib2, json, nltk
import math, random, time, pdb


In [5]:
def getPage(page):
    url_base = 'http://api.nytimes.com/svc/search/v2/articlesearch.json?'
    url_date = '&begin_date=19850814&end_date=19850816'
    url_key = '&api-key=6940176db80043d5a17343d873cfc82e'
    url_page = '&page=' + str(page)
    if page>100: raise NameError('Cannot request beyond page 100')
    url_assembled = url_base + url_date + url_page + url_key
    response = urllib2.urlopen(url_assembled)
    json_data = response.read()
    data = json.loads(json_data)
    return data

In [6]:

def getNouns(data, noun_list):
    for i in range(0,9):
        sentence = data['response']['docs'][i]['lead_paragraph']
        if not(sentence == ''or sentence is None):
            text = nltk.word_tokenize(sentence)
            poses = nltk.pos_tag(text)
            prev_noun_flag = False;
            for item in poses:
                if item[1] == 'NNP':
                    if prev_noun_flag:
                        noun_list[-1]=noun_list[-1] + ' ' + item[0]
                    else:
                        noun_list.append(item[0])
                    prev_noun_flag = True
                else: prev_noun_flag = False
            return noun_list

In [7]:
data = getPage(0)
num_pages = int(math.floor(data['response']['meta']['hits']/10))-1
print str(num_pages)+' pages found'
noun_list = []
for page in range(0,num_pages):
    data = getPage(page)
    getNouns(data, noun_list) #remember lists are mutable and python passes by reference!
    print str(page),
    #time.sleep(0.1)

95 pages found


KeyError: 'lead_paragraph'

In [63]:
print str(len(noun_list))+' proper nouns/phrases found, such as:'
for count in range(1,10):
    print noun_list[random.randrange(0, len(noun_list), 1)]

677 proper nouns/phrases found, such as:
Atlantic
Coca-Cola
House Foreign
Senator Gary Hart
Presley
South Street Seaport
John Walker
M-S-R Public Power Agency
Steve


In [64]:
noun_list_no_spaces = list(noun_list)
noun_list_no_spaces[:]=[item.encode('ascii').replace(' ','') for item in noun_list_no_spaces] 


In [65]:
string_of_nouns = ' '.join(noun_list_no_spaces)

In [66]:
# SAVE OUT THE DATA SO WE DON'T HAVE TO KEEP RE-POLLING
import pickle
outfile = open('string_nouns.txt','w')
pickle.dump(string_of_nouns,outfile)

In [67]:
# LOAD BACK IN DATA
import pickle
outfile = open('string_nouns.txt','r')
string_of_nouns = pickle.load(outfile)

In [68]:
# The pil way (if you don't have matplotlib)
from PIL import Image
import numpy as np
from wordcloud import WordCloud, ImageColorGenerator
from os import path


leo_mask = np.array(Image.open("leo_paint_color.png"))

wc = WordCloud(background_color="white", max_words=2000, mask=leo_mask, max_font_size=24)
wc.generate(string_of_nouns)

#recolor
image_colors = ImageColorGenerator(leo_mask)
wc.recolor(color_func=image_colors)

#save
wc.to_file("leo_out.png")

#show
image = wc.to_image()
image.show()



#TODO Fix wrongly detected nouns

#TODO why freezing with matplotlib??

        # Display the generated image:
        # the matplotlib way:
        import matplotlib.pyplot as plt
        plt.imshow(wordcloud)
        plt.axis("off")
        plt.show()

        # take relative word frequencies into account, lower max_font_size
        wordcloud = WordCloud(max_font_size=40, relative_scaling=.5).generate(string_of_nouns)
        plt.figure()
        plt.imshow(wordcloud)
        plt.axis("off")
        

        
        ### NOTES

#nltk.download('averaged_perceptron_tagger') # need 'punkt','averaged_perceptron_tagger'
#from __future__ import unicode_literals #??
